In [62]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

In [63]:
load_dotenv()

True

In [64]:
model = ChatGoogleGenerativeAI(
    model = os.getenv("GEMINI_MODEL"),
    api_key = os.getenv("Google_API_KEY")
)

In [65]:
# create a state

class LLMState(TypedDict):
    question: str
    answer: str
    

In [68]:
def llm_qa(state: LLMState) -> LLMState:
    
    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f"Answer the following question{question}"

    # ask that question to the llm
    answer = model.invoke(prompt).content


    # update the answer in the state
    state['answer'] = answer

    return state

In [69]:
# create graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa',llm_qa)

# add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

# compile
workflow = graph.compile()


# execute
intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'question': 'How far is moon from the earth?', 'answer': "The distance between the Earth and the Moon varies because the Moon's orbit is an ellipse, not a perfect circle.\n\n*   **Average distance:** 384,400 kilometers (238,900 miles)\n*   **Closest point (perigee):** Around 363,104 kilometers (225,623 miles)\n*   **Farthest point (apogee):** Around 405,696 kilometers (252,088 miles)"}
